In [1]:
import datetime
import greenbutton_objects
import greenbutton_objects.parse
import pandas as pd

In [9]:
filename = r'C:\Users\bmirletz\source\repos\sam_dev\pysam\Examples\green_button_data_example\XCEL_Electric_15_Minute_08-31-2022_03-25-2025.xml'
#filename = r'C:\Users\bmirletz\source\repos\sam_dev\pysam\Examples\green_button_data_example\XCEL_Electric_15_Minute_11-13-2024_06-03-2025.xml'

data = greenbutton_objects.parse.parse_feed(filename)


In [11]:
mr = list(data[0].meterReadings)[0]
print(mr.intervalBlocks[0].intervalReadings[0].timePeriod.start)
print(mr.intervalBlocks[0])

2022-09-01 00:00:00+00:00
<IntervalBlock (Interval Block)>


In [12]:
df = pd.DataFrame.from_dict(mr.intervalBlocks[0].intervalReadings[:])

In [13]:
# Adapted from https://github.com/JenDobson/greenbutton/tree/master/greenbutton

def parse_reading(interval_reading):
    start = interval_reading.timePeriod.start
    duration = interval_reading.timePeriod.duration
    value = int(interval_reading.value)
    return (start, duration, value)

In [16]:
readings = []

for interval_reading in mr.intervalBlocks[0].intervalReadings:
    readings.append(parse_reading(interval_reading))

df = pd.DataFrame(readings,columns=['Start Time','Duration','Wh'])
df = df.set_index('Start Time')

In [17]:
df_23 = df.loc['2023-01-01':'2023-12-31']
df_23 = df_23.asfreq(freq='15m',  method='pad')
print(df_23)

                                 Duration   Wh
Start Time                                    
2023-01-31 00:00:00+00:00 0 days 00:15:00  430


C:\Users\bmirletz\AppData\Local\Temp\1\ipykernel_22752\129028776.py:2: FutureWarning: 'm' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_23 = df_23.asfreq(freq='15m',  method='pad')


In [19]:
df_23 = df.loc['2022-12-31':'2024-01-01']
idx = pd.period_range(min(df_23.index), max(df_23.index), freq='15min')
df_23 = df_23.reindex(idx.astype('datetime64[ns, UTC]'), method='nearest')
print(df_23.loc['2022-12-31':'2024-01-01'])

                                 Duration    Wh
2022-12-31 00:00:00+00:00 0 days 00:15:00  1036
2022-12-31 00:15:00+00:00 0 days 00:15:00   639
2022-12-31 00:30:00+00:00 0 days 00:15:00   176
2022-12-31 00:45:00+00:00 0 days 00:15:00   192
2022-12-31 01:00:00+00:00 0 days 00:15:00   157
...                                   ...   ...
2024-01-01 22:45:00+00:00 0 days 00:15:00    77
2024-01-01 23:00:00+00:00 0 days 00:15:00   104
2024-01-01 23:15:00+00:00 0 days 00:15:00    78
2024-01-01 23:30:00+00:00 0 days 00:15:00   132
2024-01-01 23:45:00+00:00 0 days 00:15:00   114

[35232 rows x 2 columns]


In [23]:
kW_data = df_23.loc['2023-01-01 07:00:00':'2024-01-01 06:59:00']['Wh'] / 250.0
print(len(kW_data))
kW_data.to_csv('15_min_load.csv')


35040


In [24]:
hourly_data = [0] * 8760
for i, v in enumerate(df_23.loc['2023-01-01 07:00:00':'2024-01-01 06:59:00']['Wh'].values):
    hourly_data[int(i / 4)] += v / 1000.0

In [27]:
print(len(hourly_data))

8760


In [28]:
idx = range(0, 8760)
df_hrly = pd.DataFrame.from_dict({'time' : idx, 'kW' : hourly_data})
df_hrly.to_csv('hourly_load.csv')